In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd drive/My Drive/avantrika/keras-autoencoder-cbir/

/content/drive/My Drive/avantrika/keras-autoencoder-cbir


In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os


In [2]:
def visualize_predictions(decoded, gt, samples=10):
    # initialize our list of output images
    outputs = None
    gt = np.squeeze(gt, axis = -1)
    # loop over our number of output samples
    for i in range(0, samples):
      # print(gt[i])
      # print(gt[i].shape, decoded[i].shape)
      # grab the original image and reconstructed image
      original = (gt[i] * 255).astype("uint8")
      recon = (decoded[i] * 255).astype("uint8")
      # stack the original and reconstructed image side-by-side
      output = np.hstack([original, recon])
      # print(output)
      # if the outputs array is empty, initialize it as the current
      # side-by-side image display
      if outputs is None:
          outputs = output

      # otherwise, vertically stack the outputs
      else:
          outputs = np.vstack([outputs, output])

    # return the output images
    return outputs


In [3]:

model_path = "/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output/autoencoder.h5"
vis_path = "/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output/recon_vis.png"
plot_path = "/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output/plot.png"

In [5]:


IMAGES_FOLDER = '/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/train'
OUTPUT = '/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output'

### Initialise empty numpy arrays

data = np.empty((0,512,512,3), dtype=np.int8)

### Read annotation file, fetch image, normalise image and array, compose data and target arrays

for root, folders, files in os.walk(IMAGES_FOLDER):
    for file in files:
        image_path = os.path.join(IMAGES_FOLDER, file)
        image = cv2.imread(image_path)/255
        image = np.expand_dims(image, axis=0)

        if image is not None:
            data = np.vstack((data, image))


### Shuffle data and target synchronously

num_samples = data.shape[0]
arr = np.arange(num_samples)
np.random.shuffle(arr)
print("num_samples", num_samples)
data = data[arr]


print(data.shape)

np.save(os.path.join(OUTPUT,'train.npy'), data)

num_samples 101
(101, 512, 512, 3)


In [6]:

IMAGES_FOLDER = '/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/test'
OUTPUT = '/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output'

### Initialise empty numpy arrays

data = np.empty((0,512,512,3), dtype=np.int8)

### Read annotation file, fetch image, normalise image and array, compose data and target arrays

for root, folders, files in os.walk(IMAGES_FOLDER):
    for file in files:
        image_path = os.path.join(IMAGES_FOLDER, file)
        image = cv2.imread(image_path)/255
        image = np.expand_dims(image, axis=0)

        if image is not None:
            data = np.vstack((data, image))


### Shuffle data and target synchronously

num_samples = data.shape[0]
arr = np.arange(num_samples)
np.random.shuffle(arr)
print("num_samples", num_samples)
data = data[arr]


print(data.shape)

np.save(os.path.join(OUTPUT,'test.npy'), data)

num_samples 25
(25, 512, 512, 3)


In [7]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 5
INIT_LR = 1e-3
BS = 32

In [8]:

trainX = np.load('/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output/train.npy')
testX = np.load('/home/harshini/workspace/keras-autoencoder-cbir-20200908T054132Z-001/keras-autoencoder-cbir/dataset/output/test.npy')
# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0


In [9]:
# construct our convolutional autoencoder
print("[INFO] building autoencoder...")
autoencoder = ConvAutoencoder.build(512, 512, 3)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mse", optimizer=opt)

# train the convolutional autoencoder
H = autoencoder.fit(
    trainX, trainX,
    validation_data=(testX, testX),
    epochs=EPOCHS,
    batch_size=BS)


[INFO] building autoencoder...


ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (101, 512, 512, 3, 1)

In [ ]:

# use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk
print("[INFO] making predictions...")
decoded = autoencoder.predict(testX)
# print(decoded)
vis = visualize_predictions(decoded, testX)
cv2.imwrite(vis_path, vis)



In [ ]:
# construct a plot that plots and saves the training history
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot_path)

# serialize the autoencoder model to disk
print("[INFO] saving autoencoder...")
autoencoder.save(model_path, save_format="h5")

In [ ]:
# USAGE
# python index_images.py --model output/autoencoder.h5 --index output/features.pickle

# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import numpy as np
import argparse
import pickle


model_path = "/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/autoencoder.h5"
index_path = "/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/features.pickle"

# load the MNIST dataset
print("[INFO] loading MNIST training split...")

trainX = np.load('/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/train.npy')
testX = np.load('/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/test.npy')

# add a channel dimension to every image in the training split, then
# scale the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
trainX = trainX.astype("float32") / 255.0

# load our autoencoder from disk
print("[INFO] loading autoencoder model...")
autoencoder = load_model(model_path)

# create the encoder model which consists of *just* the encoder
# portion of the autoencoder
encoder = Model(inputs=autoencoder.input,
	outputs=autoencoder.get_layer("encoded").output)

# quantify the contents of our input images using the encoder
print("[INFO] encoding images...")
features = encoder.predict(trainX)

# construct a dictionary that maps the index of the MNIST training
# image to its corresponding latent-space representation
indexes = list(range(0, trainX.shape[0]))
data = {"indexes": indexes, "features": features}

# write the data dictionary to disk
print("[INFO] saving index...")
f = open(index_path, "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] loading MNIST training split...
[INFO] loading autoencoder model...
[INFO] encoding images...
[INFO] saving index...


In [ ]:
# USAGE
# python search.py --model output/autoencoder.h5 --index output/index.pickle 

# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
from imutils import build_montages
import numpy as np
import argparse
import pickle
import cv2


def euclidean(a, b):
	# compute and return the euclidean distance between two vectors
	return np.linalg.norm(a - b)

def perform_search(queryFeatures, index, maxResults=64):
	# initialize our list of results
	results = []

	# loop over our index
	for i in range(0, len(index["features"])):
		# compute the euclidean distance between our query features
		# and the features for the current image in our index, then
		# update our results list with a 2-tuple consisting of the
		# computed distance and the index of the image
		d = euclidean(queryFeatures, index["features"][i])
		results.append((d, i))

	# sort the results and grab the top ones
	results = sorted(results)[:maxResults]

	# return the list of results
	return results


model_path = "/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/autoencoder.h5"
index_path = "/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/features.pickle"
sample = 10

# load the MNIST dataset
print("[INFO] loading dataset...")
trainX = np.load('/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/train.npy')
testX = np.load('/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/test.npy')

# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# load the autoencoder model and index from disk
print("[INFO] loading autoencoder and index...")
autoencoder = load_model(model_path)
index = pickle.loads(open(index_path, "rb").read())

# create the encoder model which consists of *just* the encoder
# portion of the autoencoder
encoder = Model(inputs=autoencoder.input,
	outputs=autoencoder.get_layer("encoded").output)

# quantify the contents of our input testing images using the encoder
print("[INFO] encoding testing images...")
features = encoder.predict(testX)

# randomly sample a set of testing query image indexes
queryIdxs = list(range(0, testX.shape[0]))
queryIdxs = np.random.choice(queryIdxs, size=sample,
	replace=False)

trainX = np.squeeze(trainX, axis = -1)
testX = np.squeeze(testX, axis = -1)

# loop over the testing indexes
for i in queryIdxs:
  # take the features for the current image, find all similar
  # images in our dataset, and then initialize our list of result
  # images
  queryFeatures = features[i]
  results = perform_search(queryFeatures, index, maxResults=225)
  images = []

  # loop over the results
  for (d, j) in results:
    # grab the result image, convert it back to the range
    # [0, 255], and then update the images list
    image = (trainX[j] * 255).astype("uint8")
    # image = np.dstack([image] * 3)
    images.append(image)

  # display the query image
  query = (testX[i] * 255).astype("uint8")
  # cv2.imshow("Query", query)
  cv2.imwrite("/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/smoke_testing/" + str(j) + ".jpg", query)

  # build a montage from the results and display it
  montage = build_montages(images, (512, 512), (10, 10))[0]
  cv2.imwrite("/content/drive/My Drive/avantrika/keras-autoencoder-cbir/dataset/output/smoke_testing/montage/" + str(j) + ".jpg", montage)
  # cv2.imshow("Results", montage)
  # cv2.waitKey(0)

[INFO] loading dataset...
[INFO] loading autoencoder and index...
[INFO] encoding testing images...
